In [ ]:
# 3D extension of the countception network

The implementation of the CNN is taken from J. P. Cohen, H. Z. Lo, and Y. Bengio, “Count-ception: Counting by Fully Convolutional Redundant Counting,” 2017. https://arxiv.org/abs/1703.08710

**IMPORTANT!** Ensure that notebook was started with the KERAS_BACKEND=tensorflow variable!

In [ ]:
import sys,os,time,random
import numpy as np

import matplotlib
matplotlib.use('Agg');
import matplotlib.pyplot as plt
plt.set_cmap('jet');

import tensorflow as tf

import skimage
from skimage.io import imread, imsave
import pickle
import scipy

from scipy import stats

from os import walk

In [ ]:
%matplotlib inline

In [ ]:
# KERAS stuff 
from __future__ import print_function
import keras 

from keras import backend as K

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Conv3D, MaxPooling3D
from keras.layers.convolutional import ZeroPadding2D, ZeroPadding3D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.normalization import BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import SGD

from keras.layers import Lambda
from keras.layers import Merge

from keras.models import load_model

In [ ]:
print("keras", keras.__version__)
print("tensorflow", tf.__version__)

In [ ]:
# check the backend the ordering of the channels
print(keras.backend.backend())
print(keras.backend.image_dim_ordering())
print(K.image_data_format())

In [ ]:
# Setup for the gpu: 
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

multiple_gpu_available = False
if (len(get_available_gpus()) > 1):
    multiple_gpu_available = True
print(get_available_gpus())

In [ ]:
# check the output of the command above
tf.device("/gpu:0")
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# this pone should help with the images of the large size
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allocator_type = 'BFC'
config.gpu_options.per_process_gpu_memory_fraction = 0.95

In [ ]:
# this part is necessary to set the params from the command line
import sys
import argparse

if len(sys.argv) == 3 and sys.argv[1] == "-f": #on jupyter
    sys.argv = ['']
    
parser = argparse.ArgumentParser(description='Count-ception')

parser.add_argument('-seed', type=int, nargs='?',default=0, help='random seed for split and init')
parser.add_argument('-nsamples', type=int, nargs='?',default=32, help='Number of samples (N) in train and valid')
# TODO: Is it used ? 
parser.add_argument('-stride', type=int, nargs='?',default=1, help='The args.stride at the initial layer')
parser.add_argument('-lr', type=float, nargs='?',default=0.00005, help='This will set the learning rate ')
parser.add_argument('-kern', type=str, nargs='?',default="sq", help='This can be gaus or sq')
parser.add_argument('-cov', type=float, nargs='?',default=1, help='This is the covariance when kern=gaus')
parser.add_argument('-scale', type=int, nargs='?',default=1, help='Scale the input image and labels')
parser.add_argument('-data', type=str, nargs='?',default="cells", help='Dataset folder')
parser.add_argument('-framesize', type=int, nargs='?',default=256, help='Size of the images processed at once')

args = parser.parse_args()

In [ ]:
# set the passed parameters here if you forgot them
args.framesize = 256
args.scale = 1
args.nsamples = 21

print(args)
print(keras.backend.image_data_format())

In [ ]:
# how much to extend the initial image 
patch_size_w = int(32)
patch_size_h = int(32)
patch_size_d = int(32) # in the real example this one should be 8
framesize = int(args.framesize/args.scale)
channels = int(1)
framesize_h = int(256)
framesize_w = int(256)
framesize_d = int(27) # in the real example this one should be 27
noutputs = 1
nsamples = args.nsamples

In [ ]:
paramfilename = str(args.scale) + "-" + str(patch_size_w) + "-" + args.data + "-" + args.kern + str(args.cov) + "_params-3D.p"
datasetfilename = str(args.scale) + "-" + str(patch_size_w) + "-" + str(framesize) + "-" + args.kern + str(args.stride) + "-" + args.data + "-" + str(args.cov) + "-dataset-3D.p"
print(paramfilename)
print(datasetfilename)

In [ ]:
#reproducibility
random.seed(args.seed)
np.random.seed(args.seed)
tf.set_random_seed(args.seed)

In [ ]:
# input shape is the image shape without the pathces
input_shape = (framesize_w, framesize_h, framesize_d, channels)
ext_shape = (framesize_w + patch_size_w, framesize_h + patch_size_h, framesize_d + + patch_size_d, channels)

print(input_shape)
print(ext_shape)

In [ ]:
# check the input dimensions for the network
[x + y for x, y in zip(input_shape[0:3], (patch_size_w, patch_size_h, patch_size_d))]

In [ ]:
# function to split the data between multiple GPU's
def make_parallel(model, gpu_count):
    def get_slice(data, idx, parts):
        shape = tf.shape(data)
        size = tf.concat([ shape[:1] // parts, shape[1:] ],axis=0)
        stride = tf.concat([ shape[:1] // parts, shape[1:]*0 ],axis=0)
        start = stride * idx
        return tf.slice(data, start, size)

    outputs_all = []
    for i in range(len(model.outputs)):
        outputs_all.append([])

    #Place a copy of the model on each GPU, each getting a slice of the batch
    for i in range(gpu_count):
        with tf.device('/gpu:%d' % i):
            with tf.name_scope('tower_%d' % i) as scope:

                inputs = []
                #Slice each input into a piece for processing on this GPU
                for x in model.inputs:
                    input_shape = tuple(x.get_shape().as_list())[1:]
                    slice_n = Lambda(get_slice, output_shape=input_shape, arguments={'idx':i,'parts':gpu_count})(x)
                    inputs.append(slice_n)                

                outputs = model(inputs)
                
                if not isinstance(outputs, list):
                    outputs = [outputs]
                
                #Save all the outputs for merging back together later
                for l in range(len(outputs)):
                    outputs_all[l].append(outputs[l])

    # merge outputs on CPU
    with tf.device('/cpu:0'):
        merged = []
        for outputs in outputs_all:
            merged.append(Concatenate(axis=0)(outputs))
            
        return Model(inputs=model.inputs, outputs=merged)

In [ ]:
# custom loss funciton
def mae_loss(y_true, y_pred):
    # mae_loss might be too "greedy" and train the network on artifacts
    # prediction_count2 = np.sum(y_pred / ef)
    # mae_loss = K.sum(K.abs(prediction_count2 - (y_true/ef)))
    #Mean Absolute Error is computed between each count of the count map
    l1_loss = K.abs(y_pred - y_true)
    loss = K.mean(l1_loss)    
    return loss 

In [ ]:
# custom layers (building blocks)
def ConvFactory1(data, num_filters, filter_size, stride=1, pad=(0, 0, 0), nonlinearity=LeakyReLU(alpha=0.3)):
    # data is the input tensor, leaky rely as a nonlinearity
    # the padding is done in the first layer automatically! 
    # no need to preprocess the data
    data = ZeroPadding3D(padding = pad, data_format=None, input_shape=input_shape)(data)
    data = Conv3D(filters = num_filters, kernel_size = (filter_size, filter_size, filter_size), kernel_initializer='glorot_uniform')(data)
    data = LeakyReLU(alpha=0.3)(data)
    data = BatchNormalization()(data)
    return data
    
def SimpleFactory1(data, ch_1x1, ch_3x3):
    # used for double layers 
    conv1x1 = ConvFactory1(data, filter_size=1, pad=0, num_filters=ch_1x1)
    conv3x3 = ConvFactory1(data, filter_size=3, pad=1, num_filters=ch_3x3) 
    concat = Concatenate()([conv1x1, conv3x3])
    return concat 

In [ ]:
# TODO: check again?
def create_model(input_shape):
    main_input = Input(shape=input_shape, name='main_input')
    # print (net.shape)
    net = ConvFactory1(main_input, num_filters=64, pad=(patch_size_w, patch_size_h, patch_size_d), filter_size = 3)
    print (net.shape)
    net = SimpleFactory1(net, ch_1x1 = 16, ch_3x3 = 16)
    print (net.shape)
    net = SimpleFactory1(net, ch_1x1 = 16, ch_3x3 = 32)
    print (net.shape)
    net = ConvFactory1(net, num_filters=16, filter_size = 14)
    print (net.shape)
    net = SimpleFactory1(net, ch_1x1 = 112, ch_3x3 = 48)
    print (net.shape)
    net = SimpleFactory1(net, ch_1x1 = 40, ch_3x3 = 40)
    print (net.shape)
    net = SimpleFactory1(net, ch_1x1 = 32, ch_3x3 = 96)
    print (net.shape)

    net = ConvFactory1(net, num_filters=16, filter_size = 18)
    print (net.shape) 
    net = ConvFactory1(net, num_filters=64, filter_size = 1)
    print (net.shape) 
    net = ConvFactory1(net, num_filters=64, filter_size = 1)
    print (net.shape) 
    main_output = ConvFactory1(net, filter_size=1, num_filters=1)
    print (main_output.shape)
    
    model = Model(inputs=[main_input], outputs = main_output)
    # make the model parallel for 2 gpu's
    if multiple_gpu_available:
        model = make_parallel(model, 2)
    # mean_absolute_error
    sgd = SGD(lr=args.lr, nesterov=True)
    model.compile(loss=mae_loss, optimizer=sgd, metrics=['accuracy'])
         
    return model

In [ ]:
model = create_model(input_shape = input_shape)
# model.summary()

In [ ]:
# test memory needed to run the model
def get_model_memory_usage(batch_size, model):
    import numpy as np
    from keras import backend as K

    shapes_mem_count = 0
    for l in model.layers:
        single_layer_mem = 1
        for s in l.output_shape:
            if s is None:
                continue
            single_layer_mem *= s
        shapes_mem_count += single_layer_mem

    trainable_count = np.sum([K.count_params(p) for p in set(model.trainable_weights)])
    non_trainable_count = np.sum([K.count_params(p) for p in set(model.non_trainable_weights)])

    total_memory = 4.0*batch_size*(shapes_mem_count + trainable_count + non_trainable_count)
    gbytes = np.round(total_memory / (1024.0 ** 3), 3)
    return gbytes

get_model_memory_usage(2, model)

In [ ]:
#sanity check
print("network output size should be", [x + y for x, y in zip(input_shape[0:3], (patch_size_w, patch_size_h, patch_size_d))])

In [ ]:
# TODO: FIX THE EF COEFFICENT FOR THE CASE OF CUBE 
if (args.kern == "sq"):
    ef = (((patch_size_w*patch_size_h*patch_size_d))/args.stride)
elif (args.kern == "gaus"):
    ef = 1.0
print("ef", ef)

In [ ]:
# test run
train_start_time = time.time()
# model.fit(np.zeros([1, input_shape[0], input_shape[1], input_shape[2], input_shape[3]]), 
#          np.zeros([1, input_shape[0] + patch_size_w, input_shape[1] + patch_size_h, input_shape[2] + patch_size_d, 1]))
print(time.time() - train_start_time, "sec")

train_start_time = time.time()
# model.predict();
print(time.time() - train_start_time, "sec")

In [ ]:
# THIS PART IS NOT WORKING, YOU MIGHT HAVE TO DELETE IT ALL
# fucntions to see the results 
# def genGausImage(framesize, mx, my, mz, cov=1):  
#     framesize_x = framesize[0]
#     framesize_y = framesize[1]
#     framesize_z = framesize[2]
    
#     x, y, z = np.mgrid[0:framesize_x, 0:framesize_y, 0:framesize_z]
#     pos = np.dstack((x, y, z))
#     mean = [mx, my, mz]
#     cov = [[cov, 0, 0], [0, cov, 0], [0, 0, cov]]
#     rv = scipy.stats.multivariate_normal(mean, cov).pdf(pos)
#     return rv/rv.sum()

# # TODO: FIX!
# def getDensity(width, markers):
#     gaus_img = np.zeros((width,width))
#     for k in range(width):
#         for l in range(width):
#             if (markers[k,l] > 0.5):
#                 gaus_img += genGausImage(len(markers),k-patch_size/2,l-patch_size/2,cov)
#     return gaus_img

In [ ]:
def getMarkersCells(labelPath, scale, size):  
    labs = imread(labelPath).transpose([1,2,0])
    if len(labs.shape) == 3:
        lab = labs[:,:,:]/255
    elif len(labs.shape) == 4:
        lab = labs[:,:,:,0]/255
    else:
        print("unknown label format")
    
    binsize = [scale, scale, scale]
    out = np.zeros(size)    
    for i in range(binsize[0]):
        for j in range(binsize[1]):
            for k in range(binsize[2]):
                out = np.maximum(lab[i::binsize[0], j::binsize[1], k::binsize[2]], out)
        
    print(lab.sum(),out.sum())
    assert np.allclose(lab.sum(),out.sum(), 1)
    
    return out

def getCellCountCells(markers, x, y, z, h, w, d):
    types = [0] * noutputs
    for i in range(noutputs):
        types[i] = (markers[y:y+h,x:x+w, z:z+w] == 1).sum()
        #types[i] = (markers[y:y+h,x:x+w] != -1).sum()
    return types

def getLabelsCells(markers, img_pad, base_x, base_y, base_z, stride, scale):
    
    height = int ((img_pad.shape[0])/args.stride)
    width = int ((img_pad.shape[1])/args.stride)
    depth = int ((img_pad.shape[2])/args.stride)
    
    print("label size: ", height, width, depth)
    labels = np.zeros((noutputs, height, width, depth))
    if (args.kern == "sq"):
        for y in range(0,height):
            for x in range(0,width):
                for z in range(0, depth):
                    count = getCellCountCells(markers, x*args.stride, y*args.stride, z*args.stride, patch_size_w, patch_size_h, patch_size_d)  
                    for i in range(0,noutputs):
                        labels[i][y][x][z] = count[i]

    
    elif (args.kern == "gaus"):
        # gauss is not checked in 3D
        for i in range(0,noutputs):
            labels[i] = getDensity(width, markers[base_y:base_y+width,base_x:base_x+width, base_z:base_z+depth])
    
    
    print("getLabelsCells: DONE!")
    
    count_total = getCellCountCells(markers, 0, 0, 0, framesize_h+patch_size_h, framesize_w+patch_size_w, framesize_d+patch_size_d)
    return labels, count_total

def getTrainingExampleCells(img_raw, framesize_w, framesize_h, framesize_d, labelPath, base_x,  base_y, base_z, stride, scale):
    
    img = img_raw[base_y:base_y+framesize_h, base_x:base_x+framesize_w, base_z:base_z+framesize_d]
    img_pad = np.pad(img[:,:,:], ((int ((patch_size_w)/2), int ((patch_size_w)/2)), 
                                     (int ((patch_size_h)/2), int ((patch_size_h)/2)), 
                                     (int ((patch_size_d)/2), int ((patch_size_d)/2))), "constant")
    
    markers = getMarkersCells(labelPath, scale, img_raw.shape[0:3])
    markers = markers[base_y:base_y+framesize_h, base_x:base_x+framesize_w, base_z:base_z+framesize_d]
    markers = np.pad(markers, ((patch_size_w, patch_size_w), 
                               (patch_size_h, patch_size_h), 
                               (patch_size_d, patch_size_d)), "constant", constant_values=-1)
    
    labels, count  = getLabelsCells(markers, img_pad, base_x, base_y, base_z, args.stride, scale)
    return img, labels, count

In [ ]:
# read the files from the test folder 
import glob

# prefix = '/Users/kkolyva/'
prefix = '/home/milkyklim/'
folder = prefix + 'dl-cell-counting/algorithm/data/2017-10-16-dl-data-dapi/data/run'
img_ext = '.tif'

print('Full path:', folder)

imgs = []

for filename in glob.iglob(folder + "/*dots" + img_ext):
    imgg = filename.replace("dots","cell")
    imgp = filename.replace("dots","patch")
    imgs.append([imgg,filename, imgp])
    
if len(imgs) == 0:
    print("Issue with dataset")
    sys.exit()

In [ ]:
# put some /show images/ here to see the result of processing

In [ ]:
for path in imgs: 
    if (not os.path.isfile(path[0])):
        print(path, "bad", path[0])
    if (not os.path.isfile(path[1])):
        print(path, "bad", path[1])
    if (not os.path.isfile(path[2])):
        print(path, "bad", path[2])

In [ ]:
#DEBUG: DEV:
dataset = [] 
if (os.path.isfile(datasetfilename) and False):
    print("reading", datasetfilename)
    dataset = pickle.load(open(datasetfilename, "rb" ))
else:
    dataset_x = []
    dataset_y = []
    dataset_c = []
    print(len(imgs))
    for path in imgs:
        imgPath = path[0]
        print(imgPath)
        img = imread(imgPath).transpose([1,2,0])[:, :, :, None]
        labelPath = path[1]
        markers = imread(labelPath).transpose([1,2,0])[:, :, :, None]
        patchPath = path[2]
        lab = imread(patchPath).transpose([1,2,0])[:, :, :, None]
            
        if (img.shape != input_shape) or (markers.shape != input_shape) or (lab.shape != ext_shape):
            print("!!!! Not adding image because size is" , img.shape, markers.shape, lab.shape )
        else:
            
            lab_est = lab[:].sum() / ef
            # [(l.sum()/ef).astype(np.int) for l in lab]
            print("lab_est ", lab_est)
            
            count = np.sum(markers[:]) // 255
            print("count ", count)
            
            assert np.allclose(count,lab_est, 0)
            
            
            dataset.append((img, lab, count))        
            print("img shape", img.shape, "markers shape", markers.shape, "lab.shape", lab.shape)
        
        sys.stdout.flush()
    
    print("writing", datasetfilename)
    out = open(datasetfilename, "wb",0)
    pickle.dump(dataset, out)
    print("dataset size", len(dataset))        
print("DONE")


In [ ]:
# OBSOLETE!


# dataset = []
# if (os.path.isfile(datasetfilename)):
#     print("reading", datasetfilename)
#     dataset = pickle.load(open(datasetfilename, "rb" ))
# else:
#     dataset_x = []
#     dataset_y = []
#     dataset_c = []
#     print(len(imgs))
#     for path in imgs: 

#         imgPath = path[0]
#         print(imgPath)

#         im = imread(imgPath)
#         img_raw_raw = im.transpose([1,2,0])
        
#         # img_raw = scipy.misc.imresize(img_raw_raw, (int(img_raw_raw.shape[0]/args.scale),int(img_raw_raw.shape[1]/args.scale)))
#         img_raw = img_raw_raw
#         print(img_raw_raw.shape," ->>>>", img_raw.shape)

#         labelPath = path[1]
#         for base_x in range(0,img_raw.shape[0],framesize_h):
#             for base_y in range(0,img_raw.shape[1],framesize_w):
#                 for base_z in range(0,img_raw.shape[2],framesize_d):

#                     if (img_raw.shape[1] - base_y < framesize_w) or (img_raw.shape[0] - base_x < framesize_h) or (img_raw.shape[2] - base_z < framesize_d):
#                         print("!!!! Not adding image because size is" , img_raw.shape[1] - base_y, img_raw.shape[0] - base_x, img_raw.shape[2] - baze_z)
#                         continue

#                     img, lab, count = getTrainingExampleCells(img_raw, framesize_w, framesize_h, framesize_d, labelPath, base_y, base_x, base_z, args.stride, args.scale)
#                     print("count ", count)

#                     if img.shape[0:3] != (framesize_w,framesize_h, framesize_d):
#                         print("!!!! Not adding image because size is" , img.shape[0:3])

#                     else :   
#                         lab_est = [(l.sum()/ef).astype(np.int) for l in lab]

#                         print("lab_est ", lab_est)
                        
#                         assert np.allclose(count,lab_est, 0)

#                         dataset.append((img,lab,count))

#                         print("lab_est", lab_est, "img shape", img.shape, "label shape", lab.shape)
#                         sys.stdout.flush()
                    
#         print("dataset size", len(dataset))
                    
#     print("writing", datasetfilename)
#     out = open(datasetfilename, "wb",0)
#     pickle.dump(dataset, out)
#     out.close()
# print("DONE")

In [ ]:
# grab the data from the data set
np.random.shuffle(dataset)

np_dataset_x = np.asarray([d[0] for d in dataset], dtype=np.float32)
np_dataset_y = np.asarray([d[1] for d in dataset], dtype=np.float32)
np_dataset_c = np.asarray([d[2] for d in dataset], dtype=np.float32)

np_dataset_x = np_dataset_x # .transpose((0,3,1,2))

print("np_dataset_x", np_dataset_x.shape)
print("np_dataset_y", np_dataset_y.shape)
print("np_dataset_c", np_dataset_c.shape)

In [ ]:
length = len(np_dataset_x)

# 2/3 vs 1/3 for training and validation
n = int (nsamples / 3); 

np_dataset_x_train = np_dataset_x[0:2*n]
np_dataset_y_train = np_dataset_y[0:2*n]
np_dataset_c_train = np_dataset_c[0:2*n]
print("np_dataset_x_train", len(np_dataset_x_train))

np_dataset_x_valid = np_dataset_x[2*n:3*n]
np_dataset_y_valid = np_dataset_y[2*n:3*n]
np_dataset_c_valid = np_dataset_c[2*n:3*n]
print("np_dataset_x_valid", len(np_dataset_x_valid))

np_dataset_x_test = np_dataset_x[3*n:]
np_dataset_y_test = np_dataset_y[3*n:]
np_dataset_c_test = np_dataset_c[3*n:]
print("np_dataset_x_test", len(np_dataset_x_test))

In [ ]:
# length = len(np_dataset_x)

# # 2/3 vs 1/3 for training and validation
# n = 0 # int (args.nsamples / 3); 

# np_dataset_x_train = np_dataset_x[0:1]
# np_dataset_y_train = np_dataset_y[0:1]
# np_dataset_c_train = np_dataset_c[0:1]
# print("np_dataset_x_train", len(np_dataset_x_train))

# np_dataset_x_valid = np_dataset_x[1:2]
# np_dataset_y_valid = np_dataset_y[1:2]
# np_dataset_c_valid = np_dataset_c[1:2]
# print("np_dataset_x_valid", len(np_dataset_x_valid))

# np_dataset_x_test = np_dataset_x[2:]
# np_dataset_y_test = np_dataset_y[2:]
# np_dataset_c_test = np_dataset_c[2:]
# print("np_dataset_x_test", len(np_dataset_x_test))

In [ ]:
# some stats before the run
print("number of counts total ", np_dataset_c.sum())
print("number of counts on average ", np_dataset_c.mean(), "+-", np_dataset_c.std())
print("counts min:", np_dataset_c.min(), "max:", np_dataset_c.max())

In [ ]:
# examples of the images from the training set 
n_images_show = 7
slice = 10
plt.rcParams['figure.figsize'] = (15, 5)
plt.title("Example images")
plt.imshow(np.concatenate(np_dataset_x_train[:n_images_show, :, :, slice, 0],axis=1), interpolation='none');

In [ ]:
plt.title("Example images")
slice = 10 + patch_size_d //2
plt.imshow(np.concatenate(np_dataset_y_train[:n_images_show, :, :, slice, 0],axis=1), interpolation='none');

In [ ]:
plt.rcParams['figure.figsize'] = (15, 5)
plt.title("Counts in each image")
plt.bar(range(len(np_dataset_c_train)),np_dataset_c_train);

In [ ]:
# some stats again
print("Total cells in training", np.sum(np_dataset_c_train[0:], axis=0))
print("Total cells in validation", np.sum(np_dataset_c_valid[0:], axis=0))
print("Total cells in testing", np.sum(np_dataset_c_test[0:], axis=0))

In [ ]:
directory = "network-temp/"
model_ext = "-countception-3D.h5py"

if not os.path.exists(directory):
    os.makedirs(directory)

def save_network(net, name):
    net.save(directory + str(name) + model_ext)
    
# pay attention to the loss function    
def load_network(name):
    return load_model(directory + str(name) + model_ext, custom_objects={'mae_loss': mae_loss})

In [ ]:
np_dataset_y_train.shape

In [ ]:
num_epochs = 20
batch_size = 1

model.fit(np_dataset_x_train[:2, ], np_dataset_y_train[:2,],
          epochs = num_epochs,
          batch_size = batch_size*2,
          validation_data=(np_dataset_x_valid[:2, ], np_dataset_y_valid[:2,]))

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# TODO: is this one correct ?
# test accuracy
def test_perf(dataset_x, dataset_y, dataset_c):
    testpixelerrors = []
    testerrors = []
    bs = 1
    
    # loop is unnecessary! 
    for i in range(0,len(dataset_x), bs):

        pcount = model.predict(dataset_x, batch_size=4)
        # pcount = model.predict(dataset_x,range(i,i+bs))
        pixelerr = np.abs(pcount - dataset_y[i:i+bs]).mean(axis=(2,3))
        testpixelerrors.append(pixelerr)
        
        pred_est = (pcount/(ef)).sum(axis=(1,2,3))
        err = np.abs(dataset_c[i:i+bs].flatten()-pred_est)
        
        testerrors.append(err)
    
    return np.abs(testpixelerrors).mean(), np.abs(testerrors).mean()

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
import tensorflow as tf
model = load_network("test-2-gpu")

In [ ]:
def compute_counts(dataset_x):
    # batch_size = 1    
    # batch_size not working!
    pcount = model.predict(dataset_x.transpose([0,2,3,1]), batch_size = 4)
    pred_est = np.sum(pcount / ef, axis=(1,2))   
    print(pred_est.shape)
    return pred_est

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (15, 5)
plt.title("Training Data")

pcounts = compute_counts(np_dataset_x_train)
plt.bar(np.arange(len(np_dataset_c_train))-0.1,np_dataset_c_train, width=0.5, label="Real Count");
plt.bar(np.arange(len(np_dataset_c_train))+0.1,pcounts, width=0.5,label="Predicted Count");
plt.tight_layout()
plt.legend()

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (15, 5)
plt.title("Valid Data")

pcounts = compute_counts(np_dataset_x_valid)
plt.bar(np.arange(len(np_dataset_c_valid))-0.1,np_dataset_c_valid, width=0.5, label="Real Count");
plt.bar(np.arange(len(np_dataset_c_valid))+0.1,pcounts, width=0.5,label="Predicted Count");
plt.tight_layout()
plt.legend()

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (15, 5)
plt.title("Test Data")

pcounts = compute_counts(np_dataset_x_test)
plt.bar(np.arange(len(np_dataset_c_test))-0.1,np_dataset_c_test, width=0.5, label="Real Count");
plt.bar(np.arange(len(np_dataset_c_test))+0.1,pcounts, width=0.5,label="Predicted Count");
plt.tight_layout()
plt.legend()